In [19]:
!pip install -r requirements.txt -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 MB 7.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 2.7 MB/s eta 0:00:00


In [20]:
import os
from ipyfilechooser import FileChooser

fc = FileChooser(os.path.join(os.getcwd(), '..', 'SPICE'))
fc.filter_pattern = '*.raw'
display(fc)

FileChooser(path='/mnt/c/Users/Aaron/Documents/class-d-amp/SPICE', filename='', title='', show_hidden=False, s…

In [15]:
import os

import numpy as np

from cuda import cuda, nvrtc

def ASSERT_DRV(err):
    if isinstance(err, cuda.CUresult):
        if err != cuda.CUresult.CUDA_SUCCESS:
            raise RuntimeError(f"Cuda Error: {err}")
    elif isinstance(err, nvrtc.nvrtcResult):
        if err != nvrtc.nvrtcResult.NVRTC_SUCCESS:
            raise RuntimeError(f"Nvrtc Error: {err}")
    else:
        raise RuntimeError(f"Unknown error type: {err}")

headerPaths = [
    "/usr/local/cuda/include/math_constants.h",
    "/usr/local/cuda/include/cooperative_groups.h",
    "/usr/local/cuda/include/cuda/std/complex"
]

headerNames = [
    b"math_constants.h",
    b"cooperative_groups.h",
    b"cuda/std/complex"
]

headers = []


for p in headerPaths:
    with open(p, 'rb') as h:
        headers.append(h.read())
    
with open("kernel.cu", "rb") as src:
    err, prog = nvrtc.nvrtcCreateProgram(
        src.read(),
        b"kernel.cu",
        len(headers),
        headers,
        headerNames
    )
    ASSERT_DRV(err)

opts = [
    b"--fmad=false", 
    b"--relocatable-device-code=true", 
    b"--gpu-architecture=compute_86",
    b"--include-path=/usr/local/cuda/include",
    b"--include-path=/usr/local/cuda/include/cuda/std"
]

err, = nvrtc.nvrtcCompileProgram(prog, len(opts), opts)
_, log_size = nvrtc.nvrtcGetProgramLogSize(prog)
log = b" " * log_size
_ = nvrtc.nvrtcGetProgramLog(prog, log)
if (log_size > 1): print(log.decode(encoding='utf-8'))
ASSERT_DRV(err)



err, ptx_size = nvrtc.nvrtcGetPTXSize(prog)
ASSERT_DRV(err)

ptx = b" " * ptx_size
err, = nvrtc.nvrtcGetPTX(prog, ptx)
ASSERT_DRV(err)

ImportError: cannot import name 'nvjitlinker' from 'cuda' (/mnt/c/Users/Aaron/Documents/class-d-amp/JupyterLab/.venv/lib/python3.10/site-packages/cuda/__init__.py)

In [11]:
import numpy as np

from cuda import cuda, nvrtc

err, = cuda.cuInit(0)

err, cuDevice = cuda.cuDeviceGet(0)

err, context = cuda.cuCtxCreate(0, cuDevice)

ptx = np.char.array(ptx)

err, module = cuda.cuModuleLoadData(ptx.ctypes.data)
ASSERT_DRV(err)

err, kernel = cuda.cuModuleGetFunction(module, b"kernel")
ASSERT_DRV(err)


RuntimeError: Cuda Error: 218

In [23]:
import ltspice
import numpy as np
import cupy as cp

lt = ltspice.Ltspice("../SPICE/Class D Simplified.raw")
lt.parse()

for case in range(lt.case_count):
    raw_freq = lt.get_frequency(case)
    raw_res = lt.get_data('v(fb)', case)

    cf_desc = cp.cuda.texture.ChannelFormatDescriptor(
        32, 32, 0, 0, cp.cuda.runtime.cudaChannelFormatKindFloat
    )
    
    arr = cp.cuda.texture.CUDAarray(
        cf_desc, len(raw_freq), flags=cp.cuda.runtime.cudaArrayDefault
    )

    c = np.empty(2 * len(raw_res), dtype=np.float32)
    c[0::2] = np.real(raw_res)
    c[1::2] = np.imag(raw_res)

    arr.copy_from(c)

    res_desc = cp.cuda.texture.ResourceDescriptor(
        cp.cuda.runtime.cudaResourceTypeArray,
        cuArr=arr
    )

    tex_desc = cp.cuda.texture.TextureDescriptor(
        (cp.cuda.runtime.cudaAddressModeBorder,),
        cp.cuda.runtime.cudaFilterModeLinear,
        cp.cuda.runtime.cudaReadModeElementType,
        normalizedCoords=True
    )

    tex = cp.cuda.texture.TextureObject(res_desc, tex_desc)

    h_size = 32
    f_size = 8192
    n_size = 8192

    phase_kernel = cp.RawKernel(
        fr'''
        #define TEX_OFFSET {-np.log10(raw_freq[0])}
        #define TEX_SCALE {1 / (np.log10(raw_freq[-1]) - np.log10(raw_freq[0]))}
        #define H_SIZE {h_size}
        #define F_SIZE {f_size}
        #define I complex(0., 1.)
        ''' + r'''
        #include <cupy/complex.cuh>
        #include <math_constants.h>
        #include <cooperative_groups.h>

        using namespace cooperative_groups;
        
        __device__ complex<double> calc_res(uint3 idx, const double* h, const double* f, cudaTextureObject_t tex) {
            double n = (double) idx.z + 1;

            double tex_idx = (log10(f[idx.y] * n) + TEX_OFFSET) * TEX_SCALE;
    
            float2 tex_res = tex1D<float2>(tex, tex_idx);
            
            complex<double> res = complex((double) tex_res.x, (double) tex_res.y);

            complex<double> tmp = 2. * I * CUDART_PI * n * h[idx.x];
            
            res *= (1. - exp(-tmp)) * (1. - exp(tmp)) / (2. * n);

            return res;
        }

        extern "C" __global__
        void kernel(const double* h, const double* f, cudaTextureObject_t tex, complex<double>* res_arr) {
            extern __shared__ complex<double> sh_sums[32];

            thread_block block = this_thread_block();
            
            uint3 bid = block.group_index();
            uint3 tid = block.thread_index();

            uint3 idx;
            idx.x = bid.x;
            idx.y = bid.y;
            idx.z = bid.z * 1024 + tid.x;

            complex<double> res = calc_res(idx, h, f, tex);

            // Warp level summation

            thread_block_tile<32> warp = tiled_partition<32>(block);
            
            warp.sync();

            double res_r = real(res);
            double res_i = imag(res);
            
            for (unsigned int s = warp.size() / 2; s > 0; s >>= 1) {
                res_r += warp.shfl_down(res_r, s);
                res_i += warp.shfl_down(res_i, s);
            }

            if (warp.thread_rank() == 0) sh_sums[warp.meta_group_rank()] = complex(res_r, res_i);

            // Block level summation

            block.sync();

            for (unsigned int s = warp.size() / 2; s > 0; s >>= 1) {
                if (tid.x < s) {
                    sh_sums[tid.x] += sh_sums[tid.x + s];
                }
                block.sync();
            }

            // Grid level summation

            grid_group grid = this_grid();

            grid.sync();

            #pragma unroll
            for (unsigned int s = 0; s < grid.dim_blocks().z; s += 1) {
                if (tid.x == 0 && bid.z == s) res_arr[idx.x * F_SIZE + idx.y] += sh_sums[0];
                grid.sync();
            }
        }
    
        ''', 'kernel'
    )
    
    hs = cp.linspace(0.2, 0.8, num=h_size, dtype=np.float64)
    
    omega = cp.logspace(
        np.log10(raw_freq[0]), np.log10(raw_freq[-1] / n_size), num=f_size, base=10, dtype=np.float64
    )

    res = cp.zeros((h_size * f_size,), dtype=np.complex128)
    
    phase_kernel(
        (h_size, f_size, n_size // 1024),
        (1024,),
        (hs, omega, tex, res)
    )

    res = res.reshape(h_size, f_size).sum(axis=-1)

    import matplotlib.pyplot as plt

    plt.xscale('log')
    
    for phase_res in cp.degrees(cp.angle(res)):
        plt.plot(omega.get(), phase_res.get())
        

CUDARuntimeError: cudaErrorLaunchFailure: unspecified launch failure